In [41]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib
import time
from decimal import Decimal

import ccxt
from pybit import usdt_perpetual

import lightgbm as lgb
import joblib

In [13]:
bybit = ccxt.bybit({"apiKey":"", "secret":""})
bybit.set_sandbox_mode(True)

session_auth = usdt_perpetual.HTTP(
    endpoint="https://api-testnet.bybit.com",
    api_key="",api_secret="")

In [3]:
# define functions used for data cleaning and feature engineering
def calc_features(df):
    
    open = df['open']
    high = df['high']
    low = df['low']
    close = df['close']
    volume = df['volume']
    
    orig_columns = df.columns

    hilo = (df['high'] + df['low']) / 2
    df['BBANDS_upperband'], df['BBANDS_middleband'], df['BBANDS_lowerband'] = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    df['BBANDS_upperband'] -= hilo
    df['BBANDS_middleband'] -= hilo
    df['BBANDS_lowerband'] -= hilo
    df['DEMA'] = talib.DEMA(close, timeperiod=30) - hilo
    df['EMA'] = talib.EMA(close, timeperiod=30) - hilo
    df['HT_TRENDLINE'] = talib.HT_TRENDLINE(close) - hilo
    df['KAMA'] = talib.KAMA(close, timeperiod=30) - hilo
    df['MA'] = talib.MA(close, timeperiod=30, matype=0) - hilo
    df['MIDPOINT'] = talib.MIDPOINT(close, timeperiod=14) - hilo
    df['SMA'] = talib.SMA(close, timeperiod=30) - hilo
    df['T3'] = talib.T3(close, timeperiod=5, vfactor=0) - hilo
    df['TEMA'] = talib.TEMA(close, timeperiod=30) - hilo
    df['TRIMA'] = talib.TRIMA(close, timeperiod=30) - hilo
    df['WMA'] = talib.WMA(close, timeperiod=30) - hilo

    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['ADXR'] = talib.ADXR(high, low, close, timeperiod=14)
    df['APO'] = talib.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df['AROON_aroondown'], df['AROON_aroonup'] = talib.AROON(high, low, timeperiod=14)
    df['AROONOSC'] = talib.AROONOSC(high, low, timeperiod=14)
    df['BOP'] = talib.BOP(open, high, low, close)
    df['CCI'] = talib.CCI(high, low, close, timeperiod=14)
    df['DX'] = talib.DX(high, low, close, timeperiod=14)
    df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    # skip MACDEXT MACDFIX たぶん同じなので
    df['MFI'] = talib.MFI(high, low, close, volume, timeperiod=14)
    df['MINUS_DI'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    df['MINUS_DM'] = talib.MINUS_DM(high, low, timeperiod=14)
    df['MOM'] = talib.MOM(close, timeperiod=10)
    df['PLUS_DI'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['PLUS_DM'] = talib.PLUS_DM(high, low, timeperiod=14)
    df['RSI'] = talib.RSI(close, timeperiod=14)
    df['STOCH_slowk'], df['STOCH_slowd'] = talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df['STOCHF_fastk'], df['STOCHF_fastd'] = talib.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = talib.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['TRIX'] = talib.TRIX(close, timeperiod=30)
    df['ULTOSC'] = talib.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    df['WILLR'] = talib.WILLR(high, low, close, timeperiod=14)

    df['AD'] = talib.AD(high, low, close, volume)
    df['ADOSC'] = talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
    df['OBV'] = talib.OBV(close, volume)

    df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
    df['NATR'] = talib.NATR(high, low, close, timeperiod=14)
    df['TRANGE'] = talib.TRANGE(high, low, close)

    df['HT_DCPERIOD'] = talib.HT_DCPERIOD(close)
    df['HT_DCPHASE'] = talib.HT_DCPHASE(close)
    df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(close)
    df['HT_SINE_sine'], df['HT_SINE_leadsine'] = talib.HT_SINE(close)
    df['HT_TRENDMODE'] = talib.HT_TRENDMODE(close)

    df['BETA'] = talib.BETA(high, low, timeperiod=5)
    df['CORREL'] = talib.CORREL(high, low, timeperiod=30)
    df['LINEARREG'] = talib.LINEARREG(close, timeperiod=14) - close
    df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(close, timeperiod=14)
    df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(close, timeperiod=14) - close
    df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(close, timeperiod=14)
    df['STDDEV'] = talib.STDDEV(close, timeperiod=5, nbdev=1)

    return df

In [4]:
def get_daily_vol(close, lookback=100):
    
    """
    This function is credited to Marcos Lopez de Prado. 
    
    """
    print('Calculating daily volatility for dynamic thresholds')
    
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = (pd.Series(close.index[df0 - 1], index=close.index[close.shape[0] - df0.shape[0]:]))
        
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1  # daily returns
    df1 = df0.ewm(span=lookback).std()
    return df1

In [5]:
def create_fresh_df(symbol,timeframe,limit):

    """
    The purpose of this function is to make a dataframe with columns such as features,'buy/sell_price', and 
    'Upper/Lower'. 
    
    """
    ohlcv = bybit.fetchOHLCV(symbol=symbol,timeframe=timeframe,limit=limit)
    ohlcv.insert(0,['timestamp','open','high','low','close','volume'])

    df=pd.DataFrame(ohlcv,columns=ohlcv[0])
    df = df.drop(df.index[0]).reset_index(drop=True)
    df['timestamp'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
    df['timestamp'] = pd.to_datetime(df['timestamp'],format='&Y%m%d %H:%M')
    df.set_index('timestamp',inplace=True)

    df = calc_features(df)
    df['daily vol']=get_daily_vol(df['close'], lookback=100)
                                     
    pips = 0.5
    limit_price_dist=df['ATR']*0.5
    limit_price_dist=(limit_price_dist/pips).round()*pips
    df['buy_price'] = df['close']-limit_price_dist
    df['sell_price'] = df['close']+limit_price_dist
    
    upper_bound=((df.loc[df.index[1]:,'close']*0.2*df.loc[df.index[1]:,'daily vol'])/pips).astype(np.float32).round()*pips
    lower_bound=((df.loc[df.index[1]:,'close']*0.4*df.loc[df.index[1]:,'daily vol'])/pips).astype(np.float32).round()*pips*-1
    df['Upper']=df['close']+upper_bound
    df['Lower']=df['close']+lower_bound
    
    df = df.iloc[limit-1:,:] 
                           
    return df 

In [6]:
"""
for order in open_orders:
    bybit.cancel_order(symbol='BTCUSDT',id=order['id']) 
"""

"\nfor order in open_orders:\n    bybit.cancel_order(symbol='BTCUSDT',id=order['id']) \n"

In [61]:
df=pd.read_parquet('df_perfect01')
model = lgb.LGBMRegressor(n_jobs=-1, random_state=1)
model_buy=model.fit(df[features], df['y_buy'])
joblib.dump(model_buy, 'buy_strat', compress=True) 
df

,open,high,low,close,maker fee,daily vol,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,DEMA,...,buy_executed,sell_executed,buy_exit_date,exit_price_buy,y_buy,sell_exit_date,exit_price_sell,y_sell,y_pred_buy_by_pcv,y_pred_sell_by_pcv
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-02-23 05:50:00,49963.0,50600.0,48779.5,50060.0,-0.00025,0.029728,1565.719965,416.25,-733.219965,1207.837059,...,1,1,2021-02-23 06:35:00,50094.5,0.010622,2021-02-23 06:35:00,50094.5,-0.008070,0.002672,0.000671
2021-02-23 06:20:00,50842.0,51216.5,49530.0,50700.0,-0.00025,0.029862,809.986002,-288.85,-1387.686002,425.403967,...,1,0,2021-02-23 07:05:00,49562.0,-0.012327,2021-02-23 06:35:00,NaN,0.000000,-0.001554,0.000513
2021-02-23 06:35:00,50700.0,50863.5,50005.0,50169.0,-0.00025,0.030253,469.817542,-457.05,-1383.917542,211.108081,...,1,0,2021-02-23 07:35:00,50166.5,0.010303,2021-02-23 07:05:00,NaN,0.000000,-0.002941,0.001376
2021-02-23 07:05:00,49740.0,50465.0,49474.0,49860.5,-0.00025,0.030670,911.688266,69.70,-772.288266,501.199109,...,0,1,2021-02-23 07:35:00,NaN,0.000000,2021-02-23 07:50:00,49938.5,-0.007683,-0.000994,-0.002633
2021-02-23 07:35:00,50062.5,50560.5,49606.5,50558.0,-0.00025,0.030639,811.992971,186.00,-439.992971,318.906714,...,1,0,2021-02-23 08:05:00,49178.5,-0.017285,2021-02-23 07:50:00,NaN,0.000000,0.000015,-0.000338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-16 00:20:00,22650.5,22694.0,22398.0,22555.0,0.00010,0.053747,145.796931,39.90,-65.996931,-189.536198,...,0,1,2022-06-16 00:35:00,NaN,0.000000,2022-06-16 01:05:00,22771.0,-0.001669,0.001498,0.000507
2022-06-16 00:35:00,22555.0,22877.5,22464.5,22527.5,0.00010,0.054057,10.670640,-101.20,-213.070640,-266.317196,...,0,1,2022-06-16 01:05:00,NaN,0.000000,2022-06-16 01:05:00,NaN,0.000000,0.001825,0.001219
2022-06-16 01:05:00,22667.5,23032.5,22424.5,22951.5,0.00010,0.054733,239.011717,-91.70,-422.411717,-228.163243,...,1,0,2022-06-16 01:20:00,NaN,0.000000,2022-06-16 01:20:00,NaN,0.000000,-0.000897,-0.000512


In [8]:

features = ['ADX', 'APO', 'AROONOSC', 'AROON_aroondown', 'AROON_aroonup', 'ATR',
       'BBANDS_lowerband', 'BBANDS_middleband', 'BBANDS_upperband', 'BETA',
       'CCI', 'DX', 'EMA', 'HT_TRENDMODE', 'KAMA', 'LINEARREG',
       'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'MA','MIDPOINT', 'MOM', 'RSI', 'STDDEV', 'STOCHF_fastk',
       'STOCH_slowd', 'STOCH_slowk', 'T3', 'TRIMA', 'ULTOSC', 'WILLR', 'WMA']

#Condfiguration
symbol="BTC/USDT"
order_type="limit"
side="buy"
amount=0.1


In [ ]:
# Main loop 
while True: 
    
    df=create_fresh_df(symbol='BTCUSDT',timeframe='15m',limit=200)
    print(df)
    print('A new df created')
    # if no pos, there will be two patterns : already realized pnl or not filled so gotta cancel.
    # But the conclusion is the same : throw a new limit order
    if session_auth.my_position(symbol='BTCUSDT')['result'][0]['size']==0:
        print('No position checked')
        bybit.cancel_all_orders(symbol='BTCUSDT',params={})
        y_pred=model_buy.predict(df[features].values)
        print(y_pred)
        if y_pred>0:
            price=float(df['buy_price'])
            order=bybit.createOrder(symbol,order_type,side,amount,price,{"reduce_only": False, "time_in_force": "GoodTillCancel",
                                        'take_profit':float(df['Upper']),'stop_loss':float(df['Lower'])}) 
            print('A limit order set')
    
    elif session_auth.my_position(symbol='BTCUSDT')['result'][0]['size']!=0:
        print('A position checked')
        session_auth.set_trading_stop(symbol="BTCUSDT",side="Buy",stop_loss=float(df['Lower']),take_profit=float(df['Upper']))
        
    
    print('wait for 15mins')
    time.sleep(60*15)